In [2]:
import numpy as np

import astropy.units as u
from astropy.time import Time
from astropy.coordinates import solar_system_ephemeris

from poliastro.bodies import Sun, Earth, Jupiter
from poliastro.twobody import Orbit
from poliastro.maneuver import Maneuver
from poliastro.iod import izzo
from poliastro.plotting import OrbitPlotter2D
from poliastro.util import norm

solar_system_ephemeris.set("jpl")

<ScienceState solar_system_ephemeris: 'jpl'>

In [3]:
## Initial data
# Links and sources: https://github.com/poliastro/poliastro/wiki/EuroPython:-Per-Python-ad-Astra
date_launch = Time("2018-08-05 16:25", scale="utc")
C_3 = 31.1 * u.km ** 2 / u.s ** 2
date_flyby = Time("2020-10-09 19:21", scale="utc")
date_arrival = Time("2023-07-05 03:18", scale="utc")

In [4]:
# Initial state of the Earth
ss_e0 = Orbit.from_body_ephem(Earth, date_launch)
r_e0, v_e0 = ss_e0.rv()

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2018-08-05 16:26:09.183. Use Time(..., scale='tdb') instead.



In [5]:
# State of the Earth the day of the flyby
ss_efly = Orbit.from_body_ephem(Earth, date_flyby)
r_efly, v_efly = ss_efly.rv()

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2020-10-09 19:22:09.182. Use Time(..., scale='tdb') instead.



In [6]:
# Assume that the insertion velocity is tangential to that of the Earth
dv = C_3 ** 0.5 * v_e0 / norm(v_e0)
man = Maneuver.impulse(dv)

In [7]:
# Inner Cruise 1
ic1 = ss_e0.apply_maneuver(man)
ic1.rv()
ic1.period.to(u.year)

<Quantity 2.1222518623017486 yr>

In [8]:
op = OrbitPlotter2D()

op.plot(ss_e0)
op.plot(ic1)

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/propagation.py:232: UserWarning:

Frame <class 'astropy.coordinates.builtin_frames.icrs.ICRS'> does not support 'obstime', time values were not returned



FigureWidget({
    'data': [{'hoverinfo': 'none',
              'line': {'color': 'rgb(31, 119, 180)', 'dash':…

In [9]:
# We propagate until the aphelion
ss_aph = ic1.propagate(ic1.period / 2)
ss_aph.epoch

<Time object: scale='tdb' format='iso' value=2019-08-28 06:15>

In [10]:
# Let's compute the Lambert solution to do the flyby of the Earth
time_of_flight = date_flyby - ss_aph.epoch
time_of_flight

<TimeDelta object: scale='tai' format='jd' value=408.5459758639654>

In [11]:
time_of_flight.to(u.year)

<Quantity 1.11853792159881 yr>

In [12]:
(v_aph, v_fly), = izzo.lambert(Sun.k, ss_aph.r, ss_efly.r, time_of_flight)

In [13]:
# Check the delta-V
norm(v_aph - ss_aph.v)  # Too high!

<Quantity 1.108000623235836 km / s>

In [14]:
ss_aph_post = Orbit.from_vectors(Sun, ss_aph.r, v_aph, epoch=ss_aph.epoch)
ss_junofly = Orbit.from_vectors(Sun, r_efly, v_fly, epoch=date_flyby)

In [15]:
op = OrbitPlotter2D()

op.plot(ss_e0, label="Earth")
op.plot_trajectory(ic1.sample(50, max_anomaly=180 * u.deg), label="Inner Cruise 1")
op.plot(ss_aph_post, label="Back to Earth")

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/propagation.py:232: UserWarning:

Frame <class 'astropy.coordinates.builtin_frames.icrs.ICRS'> does not support 'obstime', time values were not returned



FigureWidget({
    'data': [{'hoverinfo': 'none',
              'line': {'color': 'rgb(31, 119, 180)', 'dash':…

In [16]:
# And now, go to Jupiter!
ss_j = Orbit.from_body_ephem(Jupiter, date_arrival)
r_j, v_j = ss_j.rv()

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2023-07-05 03:19:09.184. Use Time(..., scale='tdb') instead.



In [17]:
(v_flypre, v_oip), = izzo.lambert(Sun.k, r_efly, r_j, date_arrival - date_flyby)

In [18]:
ss_oip = Orbit.from_vectors(Sun, r_j, v_oip, epoch=date_flyby)

In [21]:
ss_oip

0 x 5 AU x 17.9 deg (HCRS) orbit around Sun (☉) at epoch 2020-10-09 19:21:00.000 (UTC)

In [22]:
ss_j

5 x 5 AU x 23.2 deg (ICRS) orbit around Sun (☉) at epoch 2023-07-05 03:19:09.184 (TDB)

In [23]:
v_flypre

<Quantity [ 34.82978398, 14.22856327,  5.67773144] km / s>

In [24]:
v_oip

<Quantity [-4.50190534,-1.38685636,-0.58877956] km / s>

In [ ]:
ss_oip.plot()

In [ ]:
op = OrbitPlotter2D()

# Plotting approximation, suggestions welcome
op.plot(ss_e0, label="Earth")
op.plot_trajectory(ic1.sample(50, max_anomaly=180 * u.deg), label="Inner Cruise 1")
op.plot_trajectory(
    ss_aph_post.sample(50, min_anomaly=180 * u.deg, max_anomaly=400 * u.deg),
    label="Back to Earth",
)
op.plot_trajectory(
    ss_oip.sample(50, min_anomaly=10 * u.deg, max_anomaly=180 * u.deg),
    label="Jupiter Orbit Insertion Phase",
)
op.plot(ss_j, label="Jupiter")

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/propagation.py:232: UserWarning:

Frame <class 'astropy.coordinates.builtin_frames.icrs.ICRS'> does not support 'obstime', time values were not returned



In [ ]:
ss_oip

In [ ]:
help(poliastro.twobody.orbit.Orbit)